In [2]:
import pandas as pd
from pathlib import Path
import configparser
import platform
import os


In [3]:
def set_folder_defaults():
    if 'macOS' in platform.platform():
        # set some defaults for testing on mac
        download_dir = Path('/Users/mark/Downloads')
        user_dir = download_dir
        sharepoint_dir = download_dir

    elif 'Server' in platform.platform():
        # we're on the azure server (probably)
        user_dir = Path('Z:/python/FilesIn')

        download_dir = Path(user_dir)
        user_dir = download_dir
        sharepoint_dir = Path('Z:/python/FilesOut')

    elif os.getlogin() == 'mark_':
        # my test windows machine
        download_dir = Path('C:/Users/mark_/Downloads')
        user_dir = download_dir
        sharepoint_dir = download_dir        

    else:
        # personal one drive
        user_dir = 'C:/Users/USERNAME'

        # replace USERNAME with current logged on user
        user_dir = user_dir.replace('USERNAME', os.getlogin())

        # read in config file
        config = configparser.ConfigParser()
        config.read('user_directory.ini')

        # read in gm_dir and gm_docs from config file
        gm_dir = Path(config[os.getlogin().lower()]['gm_dir'])
        gm_docs = Path(config[os.getlogin().lower()]['gmt'])
        # this may find more than one sharepoint directory
        # sharepoint_dir = user_dir + "/" + gm_dir + "/" + gm_docs
        sharepoint_dir = Path(user_dir / gm_dir / gm_docs)

        # download_dir = os.path.join(sharepoint_dir, 'Data Shuttle', 'downloads')
        download_dir = Path(sharepoint_dir / 'Data Shuttle' / 'downloads')

    return sharepoint_dir, download_dir, user_dir

In [4]:
def find_files(download_dir, search='*'):
    # find any changed files changed in past 2hrs in the downloads directory
    dirpath = download_dir
    files = []
    for p, ds, fs in os.walk(dirpath):
        for fn in fs:
            if search in fn:
                # was using this to filter what filenames to find
                filepath = os.path.join(p, fn)
                files.append(filepath)

    return files

In [32]:
# this gets called if running from this script.  
if __name__ == '__main__':

    sharepoint_dir, download_dir, user_dir = set_folder_defaults()
    files = find_files(download_dir / 'T48E', 'Z00001')
    # reverse the sort order
    files.sort(reverse=True)
    # just take the last x files from the top, being the latest
    files = files[:5]
    # resort from old at top to latest at bottom
    files.sort(reverse=False)

    dict_df = {}

    for file in files:
        with open(file, "rb") as f:
            # reading in the historic excel files
            df = pd.read_excel(f, parse_dates=True)
            f.close()
            # if percent_missing row is there get rid of it
            df = df[df.orig_sort != 'percent_missing']
            # format last export datea as date only
            df['Last Export Date'] = pd.to_datetime(df['Last Export Date']).dt.date
            last_export_date = df['Last Export Date'].head(1)
            



,BOM COUNT,Matching Key,Last Export Date,orig_sort,Function Group,System,Sub System,Level_4_Parent,Level,Title,...,Part Thickness,Part Type,Phase Combined,Program Gateway,Project,Source Code Name,Symmetry,cumcount,has_child,part_number_length
1,NaN,T48E-01-Z00001,2024-07-29,0,MASTER PRODUCT 5 DOOR,NaN,NaN,NaN,0.0,T48e-01-Z00001,...,NaN,NaN,NaN,NaN,NaN,SYS - System Structure,NO,NaN,1.0,14.0
2,NaN,T48E-01-01T48E-01-Z00001,2024-07-29,1,BODY STRUCTURES,NaN,NaN,NaN,1.0,T48E-01-01,...,NaN,Assembly,NaN,NaN,T48E,SYS - System Structure,NO,NaN,1.0,10.0
3,NaN,T48E-01-A01T48E-01-01,2024-07-29,2,BODY STRUCTURES,STRUCTURE SYSTEMS,NaN,NaN,2.0,T48E-01-A01,...,NaN,Assembly,NaN,NaN,T48E,SYS - System Structure,NO,NaN,1.0,11.0
4,NaN,T48E-01-A01-01T48E-01-A01,2024-07-29,3,BODY STRUCTURES,STRUCTURE SYSTEMS,OCCUPANT CELL,NaN,3.0,T48E-01-A01-01,...,NaN,Assembly,NaN,NaN,T48E,SYS - System Structure,NO,NaN,1.0,14.0
5,NaN,T48E-A12345T48E-01-A01-01,2024-07-29,4,BODY STRUCTURES,STRUCTURE SYSTEMS,OCCUPANT CELL,T48E-A12345,4.0,T48E-A12345,...,NaN,Assembly,Prototype Design | Prototype Tooling | Prototy...,XP,T48E,AIH - Assembled In House,NO,NaN,1.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7278,NaN,T48E-A12840T48E-A11718,2024-07-29,11544,BODY EXTERIORS,EXTERIOR SYSTEMS,GLAZING,T48E-A11716,6.0,T48E-A12840,...,NaN,NaN,NaN,NaN,T48E,NaN,RH,NaN,0.0,11.0
7279,NaN,T48E-A12842T48E-A11718,2024-07-29,11545,BODY EXTERIORS,EXTERIOR SYSTEMS,GLAZING,T48E-A11716,6.0,T48E-A12842,...,NaN,NaN,NaN,NaN,T48E,NaN,RH,NaN,0.0,11.0
7280,NaN,T48E-A12844T48E-A11718,2024-07-29,11546,BODY EXTERIORS,EXTERIOR SYSTEMS,GLAZING,T48E-A11716,6.0,T48E-A12844,...,NaN,NaN,NaN,NaN,T48E,NaN,RH,NaN,0.0,11.0
7281,NaN,T48E-A12846T48E-A11718,2024-07-29,11547,BODY EXTERIORS,EXTERIOR SYSTEMS,GLAZING,T48E-A11716,6.0,T48E-A12846,...,NaN,NaN,NaN,NaN,T48E,NaN,RH,NaN,0.0,11.0


In [29]:
df

1       2024-07-29
2       2024-07-29
3       2024-07-29
4       2024-07-29
5       2024-07-29
           ...    
7278    2024-07-29
7279    2024-07-29
7280    2024-07-29
7281    2024-07-29
7282    2024-07-29
Name: Last Export Date, Length: 7282, dtype: object